# Imports, Initializations

In [1]:
import os
from collections import defaultdict
from bs4 import BeautifulSoup
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from helpers import load_data_files, EldenRingItemMap

In [2]:
path="Carian-Archive-main/GameText/GR/data/INTERROOT_win64/msg/engUS/"

# Process data, establish vector database

In [4]:
data_files = load_data_files(path)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/html/parser.py:171: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [103]:
item_map = EldenRingItemMap()

for file_name, file in data_files.items():
    item_map.process_file(file_name, file)

None None
Gem Caption
Accessory Name
Petition for Help
Broken Finger Stalker Contract
Crimson Amber Medallion
Crimson Amber Medallion +1
Crimson Amber Medallion +2
Cerulean Amber Medallion
Cerulean Amber Medallion +1
Cerulean Amber Medallion +2
Viridian Amber Medallion
Viridian Amber Medallion +1
Viridian Amber Medallion +2
Arsenal Charm
Arsenal Charm +1
Great-Jar's Arsenal
Erdtree's Favor
Erdtree's Favor +1
Erdtree's Favor +2
Radagon's Scarseal
Radagon's Soreseal
Starscourge Heirloom
Prosthesis-Wearer Heirloom
Stargazer Heirloom
Two Fingers Heirloom
Silver Scarab
Gold Scarab
Moon of Nokstella
Green Turtle Talisman
Stalwart Horn Charm
Stalwart Horn Charm +1
Immunizing Horn Charm
Immunizing Horn Charm +1
Clarifying Horn Charm
Clarifying Horn Charm +1
Prince of Death's Pustule
Prince of Death's Cyst
Mottled Necklace
Mottled Necklace +1
Bull-Goat's Talisman
Marika's Scarseal
Marika's Soreseal
Warrior Jar Shard
Shard of Alexander
Millicent's Prosthesis
Magic Scorpion Charm
Lightning Scorpi

# Langchain stuff

In [105]:
embeddings_model = OpenAIEmbeddings(model='text-embedding-3-large')

In [106]:
simplest_items_as_doc = []
for group, items in item_map.items.items():
    for item_id, item in items.items():
        item_text = f'''Data dump from an item the player can find in the game Elden Ring:
==============================
Item Type: {group}
--------------------------
'''
        for key, value in item.items():
            item_text += f'''{key}: {value}'''
            item_text += "\n--------------------------\n"
        item_doc = Document(item_text)
        simplest_items_as_doc.append(item_doc)

In [110]:
print(simplest_items_as_doc[0].page_content)

Data dump from an item the player can find in the game Elden Ring:
Item Type: Accessory
--------------------------
Caption: A medallion with viridian amber inlaid.
Boosts maximum stamina.

The Erdtree's old sap becomes amber, treasured as the most precious of jewels in the age of Godfrey, the first Elden Lord.

A primordial life energy resides inside.
--------------------------



In [114]:
from langchain_community.vectorstores import FAISS

In [115]:
embedder = OpenAIEmbeddings()

In [118]:
db = FAISS.from_documents(simplest_items_as_doc, embedder)

In [119]:
db.distance_strategy = 'cosine'

In [120]:
db.save_local('simplest_items_db')

In [121]:
def search(query, k=4):
    preamble_string = f'''Answer the following question about the lore of the game Elden Ring, using information provided from the game's item descriptions.
    Question:
    '''
    return db.similarity_search_with_score(preamble_string + query, k)

In [122]:
def search_and_print(query, k=4):
    docs = search(query, k)
    for doc, score in docs:
        print(f"score: {score}")
        print("doc:")
        print(doc.page_content)

In [123]:
search_and_print("What is the primevial current?", k=20)

score: 0.29887327551841736
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Accessory
--------------------------
Caption: A talisman depicting a school of graven mages, the nightmare of the academy.

Raises potency of sorceries.

The primeval current is a forbidden tradition of glintstone sorcery. To those who cleave to its teachings, the act of collecting sorcerers to fashion them into the seeds of stars is but another path of scientific inquiry.
--------------------------

score: 0.32399657368659973
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Protector
--------------------------
Caption: Crown of Azur, primeval current sorcerer, set with a prominent blue-green glintstone.

This crown replaced Azur's brain and skull altogether, and now, removed from his body, it is all but dead. What power remains within raises the potency of Azur's primeval current sorceries at the cost of additional FP consumption.
------------

In [ ]:
search_and_print("Where does glintstone come from?", k=20)

NameError: name 'search_and_print' is not defined

In [ ]:
search_and_print("Why is Ranni a demigod?", k=20)

score: 0.32297423481941223
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Miniature Ranni
--------------------------
Caption: A doll resembling Ranni the Witch.
From head to toe every detail is perfect.

The chilliness is gone, feeling now like an empty husk.

There is no response.
--------------------------
Info: Doll resembling Ranni the Witch
--------------------------

score: 0.3233301639556885
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Miniature Ranni
--------------------------
Caption: A doll resembling Ranni the Witch.
From head to toe every detail is perfect.

This unresponsive doll seems pleasantly cool.
--------------------------
Info: Doll resembling Ranni the Witch
--------------------------

score: 0.330355703830719
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
------------------

In [ ]:
search_and_print("Where did the Crystalians come from, and what are they, exactly?", k=20)

score: 0.300118088722229
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Name: Crystal Sword
--------------------------
Caption: Sword fashioned from pure crystal; a deed impossible for a human. Enwreathed with powerful magic, its attack scales with intelligence. The inscrutable Crystalians have but one clear purpose; to safeguard their crystals unto the end. One theory posits that they yearn for the return of their creator who will carve for them new brethren.
--------------------------

score: 0.305898517370224
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Name: Crystal Spear
--------------------------
Caption: Spear fashioned from pure crystal; a deed impossible for a human. Enwreathed with powerful magic, its attack scales with intelligence. The inscrutable Crystalians have but one clear purpose; to safeguard their crystals unto the end. One th

In [ ]:
search_and_print("What is the significance of the name 'Elden Ring'?", k=20)

score: 0.3213297724723816
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: The Ring
--------------------------

score: 0.3213297724723816
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: The Ring
--------------------------

score: 0.3376530408859253
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Name: Ringed Finger
--------------------------
Caption: Bludgeon made of an enormous finger sheathed in several heavy rings. Thought to have been cut from an ancestor of the Fingercreeper.

Some life yet remains in this legacy of an ancient act of blasphemy, as evidenced by the barely perceptible warmth it still exudes.
--------------------------

score: 0.3411431908607483
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
---------------------

In [ ]:
search_and_print("Origin of the ants in the eternal cities", k=20)

score: 0.3463672399520874
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Name: Ant's Skull Plate
--------------------------
Caption: Huge head of one of the giant ants which inhabit the two underground rivers, used without modification as a shield. Excels at repelling enemy attacks.

Giant ants are venomous creatures, granting a boost to immunity when wielding this shield.
--------------------------

score: 0.36932411789894104
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Formic Rock
--------------------------
Info2: Found near Ainsel River and other places where giant ants live
--------------------------
Caption: Rock formed from solidified giant ant venom. Highly acidic. Material used for crafting items.

Found near Ainsel River and other places where giant ants live.
--------------------------
Info: Material used for crafting items
-------

In [ ]:
search_and_print("Who's the most powerful character?", k=20)

score: 0.3841433823108673
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Effect: Attack power rises as attacks continue
--------------------------

score: 0.39214813709259033
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Name: Crystal Sword
--------------------------
Caption: Sword fashioned from pure crystal; a deed impossible for a human. Enwreathed with powerful magic, its attack scales with intelligence. The inscrutable Crystalians have but one clear purpose; to safeguard their crystals unto the end. One theory posits that they yearn for the return of their creator who will carve for them new brethren.
--------------------------

score: 0.3925646245479584
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Protector
--------------------------
Info: Armor of Godfrey, the first Elden Lord
--------------------------